In [1]:
import pandas as pd

In [2]:
# Load the original CSV file
file_path = "SurfSpot_SST(Monthly).csv"  # Replace with the path to your file
df = pd.read_csv(file_path)

In [3]:
# Ensure the 'Month' column is in datetime format
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')

In [4]:
# Preserve the original surf spot order
df['Surf Spot Order'] = df['Surf Spot'].factorize()[0]  # Keep track of the original order


In [5]:
# Create a new DataFrame with daily data
daily_data = []
for surf_spot, group in df.groupby('Surf Spot', sort=False):  # sort=False preserves the original order
    # Generate a daily date range for the surf spot
    start_date = group['Month'].min()
    end_date = (group['Month'].max() + pd.offsets.MonthEnd(0))  # Ensure the range includes the full month
    daily_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    # Interpolate SST data for daily values
    group = group.set_index('Month').reindex(daily_date_range).rename_axis('Date').reset_index()
    group['Surf Spot'] = surf_spot
    group['SST (°C)'] = group['SST (°C)'].interpolate(method='linear')

    daily_data.append(group)

# Combine all surf spot data into one DataFrame
daily_df = pd.concat(daily_data, ignore_index=True)


In [6]:
# Restore the original surf spot order
daily_df['Surf Spot Order'] = daily_df['Surf Spot'].map(
    dict(zip(df['Surf Spot'], df['Surf Spot Order']))
)
daily_df = daily_df.sort_values(by=['Surf Spot Order', 'Date']).drop(columns=['Surf Spot Order'])


In [7]:
# Save the result to a new CSV file
output_path = "surf_spots_daily_sst.csv"
daily_df.to_csv(output_path, index=False)

print(f"Daily SST data saved to {output_path}")

Daily SST data saved to surf_spots_daily_sst.csv
